# Analyzing Unstructured Data with AISQL and Cortex LLM Functions

## Summary

This document explores how to leverages Snowflake's AISQL & Cortex LLM functions to analyze customer support call transcripts. The script performs various analyses including sentiment analysis, device categorization, issue extraction, resolution determination, and customer service rating. The results are organized into dynamic tables that automatically refresh when source data changes.

### Setting the Session Context


In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
print(session)


## Raw Data

Let's look at the raw data in the underlying table first so we know what we are querying.
- First we will look at the raw JSON files within the Snowflake stage
- Next we will look at the JSON files parsed into a structured table

In [ ]:
 LIST @MED_DEVICE_TRANSCRIPTS.DATA_PREP.CALL_DATA_NEW;

In [ ]:
SELECT * FROM parsed_transcripts LIMIT 10

## Script Components

### 1. Individual AISQL & Cortex LLM Function Queries

The script begins with several standalone queries that demonstrate individual functions:

#### Transcript Summarization
This query uses the `SUMMARIZE` function to generate concise summaries of each transcript.(https://docs.snowflake.com/en/sql-reference/functions/summarize-snowflake-cortex)

In [ ]:
SELECT
  conversation_id,
  transcript,
  SNOWFLAKE.CORTEX.SUMMARIZE(transcript) as transcript_summary
FROM parsed_transcripts
LIMIT 10;

## AI_Summarize_AGG
Summarizes a column of text data. For example, AI_SUMMARIZE_AGG(churn_reason) will return a summary of the churn_reason column.Unlike AI_COMPLETE and SUMMARIZE (SNOWFLAKE.CORTEX), this function supports datasets larger than the maximum language model context window. (https://docs.snowflake.com/en/sql-reference/functions/ai_summarize_agg)

In [ ]:
SELECT
  AGENT_NAME,
  AI_SUMMARIZE_AGG('How well has this agent done providig customer service in 25 words or less' || transcript) as agent_transcript_summary
FROM parsed_transcripts
GROUP BY AGENT_NAME;

#### Sentiment Analysis
This query uses the `SENTIMENT` function to analyze the emotional tone of each transcript and categorize it as Positive, Negative, or Neutral based on score thresholds.(https://docs.snowflake.com/en/sql-reference/functions/sentiment-snowflake-cortex)

In [ ]:
SELECT
  conversation_id,
  SNOWFLAKE.CORTEX.SENTIMENT(transcript) as sentiment_score,
  CASE
    WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript) > 0.33 THEN 'Positive'
    WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript) < -0.33 THEN 'Negative'
    ELSE 'Neutral'
  END as sentiment_category
FROM parsed_transcripts
LIMIT 10;

#### Sentiment Analysis - AI_SENTIMENT
This query uses the `AI_SENTIMENT` function to analyze the emotional tone of each transcript by various categories like 'agent interaction', 'problem resolution' and 'overall' categories.  You can still use the `SENTIMENT` functioin in the same query to categorize the overall interaction as Positive, Negative, or Neutral based on score thresholds.(https://docs.snowflake.com/en/sql-reference/functions/ai_sentiment)


In [ ]:
SELECT
  conversation_id,
  AI_SENTIMENT(transcript, ['agent interaction', 'problem resolution', 'overall']) as sentiment_score,
  CASE
    WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript) > 0.33 THEN 'Positive'
    WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript) < -0.33 THEN 'Negative'
    ELSE 'Neutral'
  END as sentiment_category
FROM parsed_transcripts
LIMIT 10;

#### Device Categorization
This query uses the `CLASSIFY_TEXT` function to categorize each transcript into one of several medical device categories. (https://docs.snowflake.com/en/sql-reference/functions/classify_text-snowflake-cortex)

In [ ]:
SELECT
  conversation_id,
  SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
    transcript, 
    ['Diabetes', 'Respiratory', 'Mobility', 'Urology', 'Pain Management', 'Monitoring', 'Orthopedic', 'Nutrition', 'Infusion', 'Wound Care']
    )['label'] as device_category
FROM parsed_transcripts
LIMIT 10;

#### Device Categorization - AI_CLASSIFY
This query uses the `AI_CLASSIFY` function to categorize each transcript into one of several medical device categories. (https://docs.snowflake.com/en/sql-reference/functions/ai_classify)

In [ ]:
SELECT
  conversation_id,
  AI_CLASSIFY(
    transcript, 
    ['Diabetes', 'Respiratory', 'Mobility', 'Urology', 'Pain Management', 'Monitoring', 'Orthopedic', 'Nutrition', 'Infusion', 'Wound Care'], 
    {'output_mode': 'multi'}
  ) as device_category,
  ARRAY_TO_STRING(device_category::VARIANT:labels, ', ') AS parsed_device_category
FROM parsed_transcripts
LIMIT 10;

#### Main Issue Extraction
This query uses the `EXTRACT_ANSWER` function to identify the main issue discussed in each transcript, returning the result as a JSON object containing the answer and a confidence score. (https://docs.snowflake.com/en/sql-reference/functions/extract_answer-snowflake-cortex)

In [ ]:
SELECT
  conversation_id,
  SNOWFLAKE.CORTEX.EXTRACT_ANSWER(transcript, 'What is the main issue?') as main_issue_json
FROM parsed_transcripts
LIMIT 10;

#### Resolution Determination
This query uses the `COMPLETE` function with a system prompt to determine if the customer's issue was resolved, categorizing it as Resolved, Unresolved, or Partial, with a brief explanation. (https://docs.snowflake.com/en/sql-reference/functions/complete-snowflake-cortex)

In [ ]:
SELECT
  conversation_id,
  SNOWFLAKE.CORTEX.COMPLETE(
    'mistral-large2',
    [
      {'role': 'system', 'content': 'You are a customer service quality analyst. 
        Analyze customer service transcripts and determine if the customer\'s issue was resolved. 
        Respond with exactly one word ("Resolved", "Unresolved", or "Partial") followed by a colon and 10 words or less explaining why.'},
      {'role': 'user', 'content': transcript}
    ],
    {'temperature': 0, 'max_tokens': 25}
  )['choices'][0]['messages']::STRING as resolution_with_reason
FROM parsed_transcripts
LIMIT 10;

#### Customer Service Rating
This query uses the `COMPLETE` function to rate the customer service experience on a scale of 0-10, with a brief explanation for the rating.

In [ ]:
SELECT
  conversation_id,
  SNOWFLAKE.CORTEX.COMPLETE(
    'claude-4-sonnet',
    CONCAT('Rate the customer service experience from 0 to 10, with 0 being very poor support without       resolution and 10 being highly supportive and complete resolution of the issue and a completely happy customer. Return the results with a single integer for the rating followed by a colon and then a reason for the rating. The reason should be 25 words or less.', transcript)
  ) as customer_service_rating
FROM parsed_transcripts
LIMIT 10;

#### Customer Service Rating - AI_COMPLETE
This query uses the `AI_COMPLETE` function to rate the customer service experience on a scale of 0-10, with a brief explanation for the rating. (https://docs.snowflake.com/en/sql-reference/functions/ai_complete-single-string)

In [ ]:
SELECT
  conversation_id,
  AI_COMPLETE(
    model => 'claude-4-sonnet',
    prompt => CONCAT('Rate the customer service experience from 0 to 10, with 0 being very poor support without resolution and 10 being highly supportive and complete resolution of the issue and a completely happy customer. The reason should be 25 words or less.', transcript),
    response_format => {
        'type':'json',
        'schema':{'type':'object','properties' : {'service_rating':{'type':'object','properties':
        {'rating':{'type': 'number'},'reason':{'type':'string'}}}}}
    }
  ) as customer_service_rating
FROM parsed_transcripts
LIMIT 10;

#### Transcript summary using AI_AGG
This query uses the `AI_AGG` function to provide a summary that includes the device, the main problem and how it was resolved. (https://docs.snowflake.com/en/sql-reference/functions/ai_agg#examples)

In [ ]:
SELECT
    conversation_id,
    AI_AGG(transcript, 'Provide a summary that includes the device, the main problem and how it was resolved') as ISSUE_RESOLUTION
FROM parsed_transcripts
GROUP BY 1
LIMIT 10;

### 2. Combined Analysis Query

The script includes a query that combines all the Cortex LLM functions into a single result set:

In [ ]:
SELECT
  source,
  conversation_id,
  start_time,
  end_time,
  agent_name,
  customer_name,
  transcript,
  SNOWFLAKE.CORTEX.SUMMARIZE(transcript) as transcript_summary,
  SNOWFLAKE.CORTEX.SENTIMENT(transcript) as sentiment_score,
  CASE
    WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript) > 0.33 THEN 'Positive'
    WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript) < -0.33 THEN 'Negative'
    ELSE 'Neutral'
  END as sentiment_category,
  SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
    transcript, 
    ['Diabetes', 'Respiratory', 'Mobility', 'Urology', 'Pain Management', 'Monitoring', 'Orthopedic', 'Nutrition', 'Infusion', 'Wound Care']
    )['label'] as device_category,
    SNOWFLAKE.CORTEX.EXTRACT_ANSWER(transcript, 'What is the main issue?') as main_issue_json,
    SNOWFLAKE.CORTEX.COMPLETE(
    'mistral-large2',
    [
      {'role': 'system', 'content': 'You are a customer service quality analyst. 
        Analyze customer service transcripts and determine if the customer\'s issue was resolved. 
        Respond with exactly one word ("Resolved", "Unresolved", or "Partial") followed by a colon and 10 words or less explaining why.'},
      {'role': 'user', 'content': transcript}
    ],
    {'temperature': 0, 'max_tokens': 25}
    )['choices'][0]['messages']::STRING as resolution_with_reason,
    SNOWFLAKE.CORTEX.COMPLETE(
    'mistral-large2',
    CONCAT('Rate the customer service experience from 0 to 10, with 0 being very poor support without resolution 
    and 10 being highly supportive and complete resolution of the issue and a completely happy customer. 
    Return the results with a single integer for the rating followed by a colon and then a reason for the rating.
    The reason should be 25 words or less.', transcript)
  ) as customer_service_rating
FROM parsed_transcripts
LIMIT 10;

### 3. Dynamic Tables

The script creates several dynamic tables that automatically refresh when source data changes:

#### Transcript Analysis Results
This dynamic table combines all the Cortex LLM functions into a single table that automatically refreshes when the source data changes.

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE transcript_analysis_results
  TARGET_LAG = 'DOWNSTREAM'
  WAREHOUSE = CORTEX_DEMO_WH
  REFRESH_MODE = 'AUTO'
AS
    SELECT
    source,
    conversation_id,
    start_time,
    end_time,
    agent_name,
    customer_name,
    transcript,
    SNOWFLAKE.CORTEX.SUMMARIZE(transcript) as transcript_summary,
    SNOWFLAKE.CORTEX.SENTIMENT(transcript) as sentiment_score,
    CASE
        WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript) > 0.33 THEN 'Positive'
        WHEN SNOWFLAKE.CORTEX.SENTIMENT(transcript) < -0.33 THEN 'Negative'
        ELSE 'Neutral'
    END as sentiment_category,
    SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
        transcript, 
        ['Diabetes', 'Respiratory', 'Mobility', 'Urology', 'Pain Management', 'Monitoring', 'Orthopedic', 'Nutrition', 'Infusion', 'Wound Care','Other']
        )['label'] as device_category,
        SNOWFLAKE.CORTEX.EXTRACT_ANSWER(transcript, 'What is the main issue?') as main_issue_json,
        SNOWFLAKE.CORTEX.COMPLETE(
        'mistral-large2',
        [
        {'role': 'system', 'content': 'You are a customer service quality analyst. 
            Analyze customer service transcripts and determine if the customer\'s issue was resolved. 
            Respond with exactly one word ("Resolved", "Unresolved", or "Partial") followed by a colon and 10 words or less explaining why.'},
        {'role': 'user', 'content': transcript}
        ],
        {'temperature': 0, 'max_tokens': 25}
        )['choices'][0]['messages']::STRING as resolution_with_reason,
        SNOWFLAKE.CORTEX.COMPLETE(
        'mistral-large2',
        CONCAT('Rate the customer service experience from 0 to 10, with 0 being very poor support without resolution 
        and 10 being highly supportive and complete resolution of the issue and a completely happy customer. 
        Return the results with a single integer for the rating followed by a colon and then a reason for the rating.
        The reason should be 25 words or less.', transcript)
        ) as customer_service_rating
    FROM parsed_transcripts;



In [ ]:
-- Query the dynamic table
SELECT * FROM transcript_analysis_results
LIMIT 10;

#### Main Issue Analysis
This dynamic table extracts the main issue answer and score from the JSON field and adds a confidence level based on the score.

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE main_issue_analysis
  TARGET_LAG = 'DOWNSTREAM'
  WAREHOUSE = CORTEX_DEMO_WH
  REFRESH_MODE = 'AUTO'
AS
  SELECT
    conversation_id,
    main_issue_json,
    main_issue_json[0]:answer::STRING as main_issue_answer,
    main_issue_json[0]:score::FLOAT as main_issue_score,
    CASE
      WHEN main_issue_json[0]:score::FLOAT >= 0.7 THEN 'High Confidence'
      WHEN main_issue_json[0]:score::FLOAT >= 0.3 THEN 'Medium Confidence'
      ELSE 'Low Confidence'
    END as main_issue_confidence_level
  FROM transcript_analysis_results;



In [ ]:
-- Query the dynamic table
SELECT * FROM main_issue_analysis LIMIT 10;

#### Resolution and Service Analysis
This dynamic table splits the resolution and customer service rating fields into separate columns for easier analysis.

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE resolution_service_analysis
  TARGET_LAG = 'DOWNSTREAM'
  WAREHOUSE = CORTEX_DEMO_WH
  REFRESH_MODE = 'AUTO'
AS
  SELECT
    conversation_id,
    resolution_with_reason,
    SPLIT_PART(resolution_with_reason, ':', 1) as resolution,
    TRIM(SPLIT_PART(resolution_with_reason, ':', 2)) as resolution_reason,
    customer_service_rating,
    SPLIT_PART(customer_service_rating, ':', 1) as service_rating,
    TRIM(SPLIT_PART(customer_service_rating, ':', 2)) as service_rating_reason
  FROM transcript_analysis_results;



In [ ]:
-- Query the resolution and customer service dynamic table
SELECT * FROM resolution_service_analysis LIMIT 10;

#### Final Combined Analysis
This final dynamic table combines all three analysis tables into a single comprehensive view, with the device_category field explicitly cast to VARCHAR for better usability.

In [ ]:
CREATE OR REPLACE DYNAMIC TABLE TRANSCRIPT_ANALYSIS_RESULTS_FINAL
  TARGET_LAG = '1 MINUTE'
  WAREHOUSE = CORTEX_DEMO_WH
  REFRESH_MODE = 'AUTO'
AS
  SELECT
    t.source,
    t.conversation_id,
    t.start_time,
    t.end_time,
    t.agent_name,
    t.customer_name,
    t.transcript,
    t.transcript_summary,
    t.sentiment_score,
    t.sentiment_category,
    t.device_category::VARCHAR as device_category,
    m.main_issue_answer,
    m.main_issue_score,
    m.main_issue_confidence_level,
    r.resolution,
    r.resolution_reason,
    r.service_rating,
    r.service_rating_reason
  FROM transcript_analysis_results t
  JOIN main_issue_analysis m ON t.conversation_id = m.conversation_id
  JOIN resolution_service_analysis r ON t.conversation_id = r.conversation_id;



In [ ]:
  -- Query the combined dynamic table
SELECT * FROM TRANSCRIPT_ANALYSIS_RESULTS_FINAL LIMIT 10;

## Usage

The dynamic tables created by this script can be used for various analytical purposes:

1. **Transcript Analysis Results**: Provides a comprehensive view of all analyses performed on each transcript.
2. **Main Issue Analysis**: Focuses on the main issues identified in each conversation, with confidence levels.
3. **Resolution and Service Analysis**: Provides insights into resolution status and customer service ratings.
4. **Final Combined Analysis**: Offers a complete view of all analyses in a single table, optimized for reporting and dashboard creation.

Each dynamic table automatically refreshes when the source data changes, ensuring that analyses are always up-to-date. 